In [73]:
import cv2 as cv
import numpy as np
import random

# 加载藻类图像和背景图像
image_algae = cv.imread('F:/github/Goodluck_babies_VisAlgae23/augmentation/random_gen/gen_augmenter_src/algaes/5Haematococcus pluvialis\83.jpg')  # 替换为藻类图像的路径
image_bg = cv.imread('F:/github/Goodluck_babies_VisAlgae23/augmentation/random_gen/gen_augmenter_src/backgrounds/back_0.jpg')  # 替换为背景图像的路径

# 定义旋转的角度
angle = 40  # 旋转45度

# 获取旋转矩阵
image_center = tuple(np.array(image_algae.shape[1::-1]) / 2)
rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)


sin_cos = rot_mat[:,1]  #从旋转矩阵中获取sin与cos
size_convert = np.mat([[sin_cos[1],sin_cos[0]],[sin_cos[0],sin_cos[1]]])    #计算形状变化的矩阵（可用于bbox的w，h）
alg_size = image_algae.shape[1::-1]
out_size = np.dot(alg_size , size_convert)
w_convert = int(out_size[:,0])  #旋转后的宽
h_convert = int(out_size[:,1])  #旋转后的高

# 旋转藻类图像
rotated_algae = cv.warpAffine(image_algae, rot_mat, (w_convert,h_convert), flags=cv.INTER_LINEAR, borderMode=cv.BORDER_REPLICATE)

# 创建旋转后的图像掩码
mask = np.zeros(rotated_algae.shape[:2], dtype=np.uint8)
mask[rotated_algae.any(axis=-1)] = 255

# 设置无缝克隆的中心位置（可能需要调整）
center = (image_bg.shape[1] // 2, image_bg.shape[0] // 2)

# 执行无缝克隆
blended_image = cv.seamlessClone(rotated_algae, image_bg, mask, center, cv.NORMAL_CLONE)

# 显示或保存结果
cv.imshow('Blended Image', blended_image)
cv.waitKey(0)
cv.destroyAllWindows()

[[0.76604444 0.64278761]
 [0.64278761 0.76604444]]
[[142.08927017 138.26830834]]
